In [42]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

%matplotlib inline

# Question

1.What are the top 3 and the bottom 3 states in terms of number of users?

2.What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically define user engagement. What the CEO cares about here is in which states users are using the product a lot/very little.

3.The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Can you give him a list of those users?


4.Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.
      
5.How would you set up a test to check whether your model works well and is improving engagement?

In [43]:
# Load Dataset

In [60]:
data = pd.read_json('/Users/check4068/Desktop/Product Sense/8.Song Challenge/song.json')
data['time_played'] = pd.to_datetime(data['time_played'])
data['user_sign_up_date'] = pd.to_datetime(data['user_sign_up_date'])
data

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00
...,...,...,...,...,...,...
3995,MKVEXMRJGF,122,Louisiana,2015-05-16,Paperback Writer,2015-06-03 08:26:26
3996,HMBEUWPBQI,76,New York,2015-05-08,Reprise / Day in the Life,2015-06-14 15:46:04
3997,KIYDZUFUJN,68,Florida,2015-05-08,Get Back,2015-06-13 13:13:11
3998,BZNJXQSTTL,138,Alabama,2015-05-17,Help!,2015-06-01 17:01:40


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 4000 non-null   object        
 1   user_id            4000 non-null   int64         
 2   user_state         4000 non-null   object        
 3   user_sign_up_date  4000 non-null   datetime64[ns]
 4   song_played        4000 non-null   object        
 5   time_played        4000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 187.6+ KB


In [46]:
# check missing values
data.isnull().sum()

id                   0
user_id              0
user_state           0
user_sign_up_date    0
song_played          0
time_played          0
dtype: int64

In [47]:
# check unique values
for name in data.columns:
    print('{0:15s} \t {1:5d}'.format(name, len(data[name].unique())))

id              	  4000
user_id         	   196
user_state      	    41
user_sign_up_date 	    20
song_played     	    97
time_played     	  3997


# Analysis

### Question 1
What are the top 3 and the bottom 3 states in terms of number of users?

In [48]:
def unique_count(x):
    """ function to count the unique user_id """
    return len(np.unique(x))

In [49]:
# create temporary dataframe for unique user count per state
state_user_count = data.groupby('user_state')['user_id'].apply(unique_count).reset_index()
state_user_count = state_user_count.rename(columns={'user_id': 'user_count'})
state_user_count = state_user_count.sort_values(by='user_count', ascending=False)

In [50]:
# get the top 3 states
state_user_count.head(3)

,user_state,user_count
25,New York,23
4,California,21
35,Texas,15


In [51]:
# get the bottom 3 states
state_user_count.tail(9)

,user_state,user_count
24,New Mexico,1
9,Idaho,1
27,North Dakota,1
6,Connecticut,1
12,Iowa,1
32,Rhode Island,1
22,Nebraska,1
2,Arizona,1
13,Kansas,1


### Question 2
2.What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically define user engagement.

I define 'average play event per hour' as a metric to measure user engagement of a state.

In [52]:
def count_by_state(df):
    """ all data in df come from the same state """
    total_played = df.shape[0]
    first_play_dt = df.time_played.min()
    last_play_dt = df.time_played.max()
    duration = last_play_dt - first_play_dt
    duration_hours = duration.total_seconds()/60.0
    return pd.Series([first_play_dt,last_play_dt, duration,duration_hours, total_played],
                     index=["first_play_dt",'last_play_dt','duration','duration_hours','total_played'])

In [53]:
counts_by_states = data.groupby("user_state").apply(count_by_state)

In [54]:
counts_by_states.head()

,first_play_dt,last_play_dt,duration,duration_hours,total_played
user_state,,,,,
Alabama,2015-06-01 14:17:56,2015-06-28 14:48:55,27 days 00:30:59,38910.983333,104
Alaska,2015-06-01 18:48:18,2015-06-28 22:58:23,27 days 04:10:05,39130.083333,58
Arizona,2015-06-01 13:05:17,2015-06-28 19:23:45,27 days 06:18:28,39258.466667,22
Arkansas,2015-06-01 14:07:37,2015-06-28 17:40:11,27 days 03:32:34,39092.566667,34
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,39722.783333,425


In [55]:
counts_by_states["hr_average"] = counts_by_states.total_played/counts_by_states.duration_hours
counts_by_states.sort_values(by="hr_average",ascending=False,inplace=True)
counts_by_states

,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
New York,2015-06-01 06:14:45,2015-06-28 21:36:40,27 days 15:21:55,39801.916667,469,0.011783
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,39722.783333,425,0.010699
Texas,2015-06-01 06:09:04,2015-06-28 20:28:35,27 days 14:19:31,39739.516667,230,0.005788
Ohio,2015-06-01 05:02:54,2015-06-28 22:22:25,27 days 17:19:31,39919.516667,209,0.005236
Florida,2015-06-01 09:29:39,2015-06-28 22:59:27,27 days 13:29:48,39689.800000,180,0.004535
Pennsylvania,2015-06-01 05:19:08,2015-06-28 21:44:20,27 days 16:25:12,39865.200000,179,0.004490
North Carolina,2015-06-01 12:40:31,2015-06-28 23:26:38,27 days 10:46:07,39526.116667,154,0.003896
Illinois,2015-06-01 12:15:13,2015-06-28 18:07:10,27 days 05:51:57,39231.950000,149,0.003798
Georgia,2015-06-01 06:41:36,2015-06-28 21:37:34,27 days 14:55:58,39775.966667,135,0.003394


In [56]:
print ("top 3 states in user engagement")
counts_by_states.iloc[:3]

top 3 states in user engagement


,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
New York,2015-06-01 06:14:45,2015-06-28 21:36:40,27 days 15:21:55,39801.916667,469,0.011783
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,39722.783333,425,0.010699
Texas,2015-06-01 06:09:04,2015-06-28 20:28:35,27 days 14:19:31,39739.516667,230,0.005788


In [68]:
print ("bottom 3 states in user engagement")
counts_by_states.iloc[-3:]

bottom 3 states in user engagement


,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
Connecticut,2015-06-06 19:23:58,2015-06-28 13:16:32,21 days 17:52:34,31312.566667,16,0.000511
New Mexico,2015-06-01 05:22:30,2015-06-28 13:15:58,27 days 07:53:28,39353.466667,17,0.000432
Kansas,2015-06-05 15:01:50,2015-06-27 09:02:15,21 days 18:00:25,31320.416667,8,0.000255


### Question 3
The CEO wants to send a gift to the first user who signed-up for each state. That is, the first user who signed-up from California, from Oregon, etc. Can you give him a list of those users?

In [77]:
def find_first_signup(df):
    idx = np.argmin(df.user_sign_up_date)
    return df.iloc[idx,[1,3]]

In [79]:
first_users = data.groupby("user_state").apply(find_first_signup)
first_users.sort_values(by="user_sign_up_date")

,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Texas,7,2015-05-01
Oregon,1,2015-05-01
Ohio,3,2015-05-01
North Carolina,2,2015-05-01
New Mexico,4,2015-05-01
New Jersey,6,2015-05-01
Pennsylvania,11,2015-05-02
New York,19,2015-05-02


### Question 4

Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.

In [41]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


There should be a lot of different methods for this question. The simplest method is to choose the most popular songs. In addition, k-nearest neighbor (KNN) method can also be used. More advancely, collaborative filtering method can be implemented.

Here, I implement a simple version of collaborative filtering algorithm for song recommendation. More specifically, the similarity of two songs is calculate using the number of users whole listen them together.

In [82]:
# Step 1: build the Song-User matrix
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user = (song_user > 0).astype(int)

song_user

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,1,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOUR MOTHER SHOULD KNOW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yellow Submarine,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Yesterday,1,1,0,0,0,0,0,0,1,0,...,1,1,1,1,1,1,0,0,0,1


In [34]:
# Step 2: build song-song similarity matrix
song_user_norm = normalize(song_user, axis=1)  # normalize the song-user matrix
similarity = np.dot(song_user_norm, song_user_norm.T)  # calculate the similarity matrix
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

similarity_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,Baby You're A Rich Man,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.263117,0.138675,0.14825,0.131559,0.299572,0.294174,0.098058,0.228802,0.200160,...,0.525213,0.113228,0.585429,0.278503,0.539411,0.087706,0.328897,0.377141,0.163430,0.0
A Hard Day's Night,0.263117,1.000000,0.000000,0.00000,0.100000,0.146385,0.111803,0.000000,0.000000,0.091287,...,0.305788,0.129099,0.266996,0.000000,0.157027,0.000000,0.050000,0.215003,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.138675,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.161165,0.000000,0.000000,0.182574,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.148250,0.000000,0.000000,1.00000,0.000000,0.164957,0.094491,0.125988,0.000000,0.000000,...,0.172292,0.000000,0.188044,0.097590,0.176950,0.000000,0.000000,0.103835,0.000000,0.0
Across The Universe,0.131559,0.100000,0.000000,0.00000,1.000000,0.097590,0.000000,0.000000,0.000000,0.000000,...,0.101929,0.000000,0.133498,0.000000,0.104685,0.000000,0.000000,0.061430,0.000000,0.0


In [103]:
# Step 3: find the top-k most similar songs
def find_topk(song, similarity, k=1):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    
    return df

In [104]:
# Example: find the top 10 similar song for 'A Day In The Life'
df = find_topk(song='A Day In The Life', similarity=similarity_df, k=10)
df

,Song,Similarity
0,Revolution,0.710186
1,Come Together,0.696557
2,Get Back,0.676128
3,Hello Goodbye,0.617065
4,Back In the U.S.S.R.,0.614700
5,Let It Be,0.601105
6,Hey Jude,0.599145
7,While My Guitar Gently Weeps,0.585429
8,Lucy In The Sky With Diamonds,0.577453
9,Here Comes The Sun,0.561283


### Question 5

We need to perform a A/B test ([reference](https://github.com/stasi009/TakeHomeDataChallenges/blob/master/08.SongChallenge/song_challenge.ipynb)):

* randomly split users into two groups, one Control group and one Experiment group
* Control group has no recommendation strategy
* Experiment group recommend the next song
* after running some time, perform a one-tailed t-test on 'average #play per hour'
    - $H_0$: population 'average #play per hour' is same in two groups
    - $H_a$: experiment group's population 'average #play per hour' is higher than control group's